In [5]:
#install.packages('partykit',repos='http://cran.us.r-project.org')

In [4]:
library(data.table)
library(ggplot2)
library(partykit)
library(plyr)

Warning message:
"package 'partykit' was built under R version 3.5.3"Loading required package: grid
Loading required package: libcoin
Warning message:
"package 'libcoin' was built under R version 3.5.3"Loading required package: mvtnorm
Warning message:
"package 'mvtnorm' was built under R version 3.5.2"Warning message:
"package 'plyr' was built under R version 3.5.3"

In [6]:
Data=read.csv('R_Training_Counting.csv')
Data[Data[,'NumPax']<0,'NumPax']=0

In [8]:
class(eq.mob <- NumPax ~ 1 + Dprio | dday + dtime + Direction)  

curve.model <- glmtree(
        eq.mob, 
        data = Data,
        family = poisson, 
        alpha = 0.01,
        bonferroni = TRUE,
        verbose = TRUE,
        prune = "BIC",
        minsize = 10,
        breakties = TRUE,
        restart = TRUE)

[1] "formula"


-- Node 1 ---------------------------------
Number of observations: 200124

Parameter instability tests:
                  dday        dtime Direction
statistic 3.978570e+02 7.512506e+01 4.9158231
p.value   3.227567e-77 5.441538e-15 0.2354792

Best splitting variable: dday
Perform split? yes
Selected split: Friday, Saturday, Sunday | Monday, Thursday, Tuesday, Wednesday

-- Node 2 ---------------------------------
Number of observations: 69592

Parameter instability tests:
                  dday        dtime    Direction
statistic 7.528896e+01 1.764942e+02 2.825217e+01
p.value   5.193005e-15 7.062527e-38 2.199082e-06

Best splitting variable: dtime
Perform split? yes
Selected split: <= 405 | > 405

-- Node 3 ---------------------------------
Number of observations: 10787

Parameter instability tests:
                dday dtime Direction
statistic 4.70192937     0         0
p.value   0.09527721    NA        NA

Best splitting variable: dday
Perform split? no

-- Node 4 ----------------

In [9]:
Forecast_Data=read.csv('R_Test_Counting.csv')

In [10]:
forecast=predict(curve.model,newdata=Forecast_Data,type='response')

In [11]:
forecast_numeric=as.numeric(forecast)
Forecast_Data=cbind(Forecast_Data,forecast)

In [12]:
sefit=function(object,newdata) predict(object,newdata,se.fit=TRUE)$se.fit
se=predict(curve.model,Forecast_Data,type=sefit)
Forecast_Data=cbind(Forecast_Data,se)

In [13]:
write.csv(Forecast_Data,'R_Output.csv',row.names=FALSE)